In [3]:
import numpy as np
import pandas as pd

In [4]:
DATE = '2020_03_30'

data = pd.read_csv(f'countermeasures_features_{DATE}.csv').drop(columns='Unnamed: 0').fillna(0)

REPORTCOLS = {'Widespread mask usage (over 75%)':('Mask wearing',lambda x: x>=75),
              'Confirmed case isolation':('Symptomatic isolation - targeted',lambda x: x>=1),
              'Contact isolation':('Asymptomatic isolation - targeted',lambda x: x>=1),
              'Gatherings limited to 10 or less':('Gatherings banned',lambda x: (x>1) and (x<=10)),
              'Gatherings limited to 100':('Gatherings banned',lambda x: (x>10) and (x<=100)),
              'Gatherings limited to 500':('Gatherings banned',lambda x: (x>100) and (x<=500)),
              'Some businesses suspended':('Activity cancellation',lambda x:(x>=1) and (x<=2)),
              'Many businesses suspended':('Nonessential business suspension',lambda x: x>=1),
              'Schools and universities closed':('School closure',lambda x: x>=1),
              'Social distancing advised':('Public interaction reduction',lambda x: (x>=1) and (x<3)),
              'Staying home advised':('Public interaction reduction',lambda x: x>=3),
              '1 - 10 000 tests':('Testing',lambda x: (x>=1) and (x<=10000)),
              '10 000 - 100 000 tests':('Testing',lambda x: (x>10000) and (x<=1e5)),
              '100 000+ tests':('Testing',lambda x: x>1e5),
              'General curfew':('Asymptomatic isolation - blanket',lambda x: (x>=1) and (x<2)),
              'General strict isolation':('Asymptomatic isolation - blanket',lambda x: x>=2)}

for rc, (vc, f) in REPORTCOLS.items():
    data[rc] = data[vc].apply(f).astype(int)
    
outdata = data[list(REPORTCOLS.keys())+['Country','Date']]

outdata.to_csv(f'interventions_visualisation_boolean_{DATE}.csv')



FileNotFoundError: [Errno 2] File countermeasures_features_2020_03_30.csv does not exist: 'countermeasures_features_2020_03_30.csv'

In [ ]:
def gatherings(n):
    if n < 1:
        return 0
    elif n <= 10:
        return 10
    elif n <= 100:
        return 100
    elif n <= 500:
        return 500
    return 0

def tests(n):
    if n<1:
        return 0
    elif n<=1e4:
        return 1e4
    elif n<=1e5:
        return 1e5
    return 1e6

def curfew(n):
    if n < 1:
        return 0
    elif n<2:
        return 1
    return 2

REPORTCOLS1 = {'Widespread mask usage (over 75%)':('Mask wearing',lambda x: x>=75),
              'Confirmed case isolation':('Symptomatic isolation - targeted',lambda x: x>=1),
              'Contact isolation':('Asymptomatic isolation - targeted',lambda x: x>=1),
              'Gatherings limited to':('Gatherings banned',gatherings),
              'Some businesses suspended':('Activity cancellation',lambda x:(x>=1) and (x<=2)),
              'Many businesses suspended':('Nonessential business suspension',lambda x: x>=1),
              'Schools and universities closed':('School closure',lambda x: x>=1),
              'Social distancing advised':('Public interaction reduction',lambda x: (x>=1) and (x<3)),
              'Staying home advised':('Public interaction reduction',lambda x: x>=3),
              'Test number (order of magnitude)':('Testing',tests),
              'General curfew':('Asymptomatic isolation - blanket',curfew)}

data0 = data.copy()

for rc, (vc, f) in REPORTCOLS1.items():
    data0[rc] = data0[vc].apply(f).astype(int)
    
outdata = data0[list(REPORTCOLS1.keys())+['Country','Date']]
outdata.to_csv(f'interventions_visualisation_semiquantified_{DATE}.csv')